In [35]:
%%capture
%run 02_transaction.ipynb
%run 03_merkle.ipynb
%run 04_account.ipynb 

In [36]:
from objsize import get_deep_size as get_size

## Block Header

The `Header` is a summary of the most important facts about a block. It contains the following:
- `root`: Root of the Merkle Tree of TXs
- `prev_hash`: Hash of the previous block
- `number`: Block number
- `n_txs`: Number of txs included in the block
- `mined`: Does the header belong to a block that is mined
- `time`: Time of creation

The actual mining happens on the block header.

In [75]:
class Header(Hashable):
    def __init__(self, root, prev_hash, number, n_txs):
        self.root      = root
        self.prev_hash = prev_hash
        self.number    = number
        self.n_txs     = n_txs
        self.mined     = False
        self.time      = time.time()

In [76]:
bh = Header(rh(),rh(),2,10); print(bh)

root:          📣 0x7d42f75e2f...451
hash:          🔝 0xb7228036dc...4bd
prev_hash:     💒 0x2cd8a449da...9c2
number:        2
n_txs:         10
mined:         False
time:          1617662024.736254 eth


If anything in the block header changes its `hash` changes automatically as well.

In [77]:
bh_changed        = deepcopy(tx2)
bh_changed.number = 120
assert bh != bh_changed

### Info

Includes all extra informations that are irrelevant for mining and therefore are not in the block header. These are the total txs `volume` and `fees` included in a block.

In [78]:
class Info(Hashable):
    def __init__(self, txs): 
        self.volume = sum([tx.value for tx in txs])
        self.fees   = sum([tx.fee   for tx in txs])

## Block

Consists of the block header `bh` and a list of txs. To validate that the txs are correct, the merkle tree can be rebuild and checked against the root hash stored in the `bh`.

In [79]:
class Block(Hashable): 
    def __init__(self, bh, txs):
        self.info   = Info(txs)
        self.header = bh
        self.mt     = MerkleTree(txs)
        self.txs    = self.val_txs(txs)
        self.hash   = bh.hash
        
    def val_txs(self, txs):
        for tx in txs: assert val_sig(tx),       'tx signature invalid'
        assert self.are_unique(txs),             'txs include duplicates'
        assert self.mt.root == self.header.root, 'txs root hash do not match'
        return {tx.hash: tx for tx in txs}
    
    def json(self): 
        info = self.info.__dict__
        h    = self.header.__dict__
        txs  = [tx.json() for tx in self.txs.values()]
        d    = {**info, **h, 'txs': txs}
        return json.dumps(d, indent=4)
    
    def are_unique(self, txs): return len(set([tx.hash for tx in txs])) == len(txs)
    def __getitem__(self, key):return self.txs[key] 
    def __str__(self):         return (str(self.header)
                                       +'\n'+str(self.info)
                                       +'\n\ntxs:\n' 
                                       +'\n'.join(tx.smry() for tx in self.txs.values()))

Create random list of signed txs.

In [80]:
acc1,acc2 = Account(),Account()
def r_stxs(n): return [acc1.signed_tx(acc2,ri(1,9),ri(1,9)/100) for _ in range(n)]

Create block from a block header containing its txs merkle tree.

In [81]:
txs = r_stxs(4)
mt  = MerkleTree(txs)
bh  = Header(mt.root, rh(), 2, len(txs))
b   = Block(bh, txs); print(b)

root:          🔺 0xd4350a8787...53d
hash:          👰 0x0ab88de43f...d56
prev_hash:     🔓 0xad50d9b8d9...a4f
number:        2
n_txs:         4
mined:         False
time:          1617662040.733291 eth
volume:        30.0 eth
hash:          🔧 0xc13a01ba3d...405
fees:          0.11 eth

txs:
💽 0x57C7 -> 💠 0x3aB4 9.0 eth
💽 0x57C7 -> 💠 0x3aB4 9.0 eth
💽 0x57C7 -> 💠 0x3aB4 7.0 eth
💽 0x57C7 -> 💠 0x3aB4 5.0 eth


### Block JSON

Every block has a json representation that we will use for our API later.

In [82]:
block_json = b.json()

We can load this JSON string `d` and create a block object again.

In [83]:
def load_block(d):
    d  = json.loads(d)
    txs = []
    for tx in d['txs']: txs.append(load_tx(tx))
    bh = Header(d['root'],d['prev_hash'],d['number'],d['n_txs']) 
    for k,v in d.items(): setattr(bh, k, v)
    bh.hash = d['hash']
    b = Block(bh, txs)
    return b
    
b_from_json = load_block(b.json())

In [84]:
assert b == b_from_json

#### Access specific tx from block with its hash.

In [85]:
tx_hash = next(iter(b.txs.keys())); tx_hash

'0x61d9fa5548e041a4383541e53f26d3ef285cc1aa5a3f4e0042cde530f0aa226d'

In [86]:
tx = b[tx_hash]; print(tx)

fr:            💽 0x57C7c3dDf3...97c
hash:          📇 0x61d9fa5548...26d
to:            💠 0x3aB48f57D1...Cda
value:         9.0 eth
fee:           0.01 eth
nonce:         0
time:          Tue Apr  6 00:34:00 2021
signed:        True
sig:           🔎 0xa853175c0a...71b


Blocks can only contain unique txs. 

In [87]:
txs[0] = txs[1]

mt  = MerkleTree(txs)
bh  = Header(mt.root, rh(), 2, len(txs))

In [88]:
dup_detected = False
try:    Block(bh, txs)
except: dup_detected = True
assert  dup_detected